In [18]:
import os
import cv2
import numpy as np
import random


from scipy.spatial.distance import cosine
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt



In [19]:
index_of_features={}
import pickle

def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    if random.random() > 0.5:
        image = cv2.flip(image, 1)
    
    #show image
    return image

def extract_features(image):
    model = MobileNet(weights='imagenet', include_top=False, pooling='avg')
    image = preprocess_input(image)
    features = model.predict(np.expand_dims(image, axis=0))
    return features.flatten()

def normalize_features(features):
    normalized_features = (features - np.mean(features)) / np.std(features)
    return normalized_features

def retrieve_similar_images(query_features, database_features, top_k=5):
    similarity_scores=[]
    for img_path, features in database_features.items():
        similarity = 1-cosine(query_features, features)
        similarity_scores.append((img_path, similarity))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    return similarity_scores[:top_k]

def save_features(image_folder):
    image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        image = preprocess_image(image_path)
        features = extract_features(image)
        normalized_features = normalize_features(features)
        index_of_features[image_file] = normalized_features
    with open('index_of_features.pkl', 'wb') as f:
        pickle.dump(index_of_features, f)


# Image folder path
os.chdir('/Users/pait/Desktop/Coding/IR')
image_folder = "images"


# Preprocess images, extract features, normalize, and save features
# save_features(image_folder)


In [20]:

query_image_path = "images/1.jpg"
query_image = preprocess_image(query_image_path)
query_features = extract_features(query_image)
query_features_normalized = normalize_features(query_features)

with open('index_of_features.pkl', 'rb') as f:
    database_features = pickle.load(f)


# Retrieve similar images
top_indices = retrieve_similar_images(query_features_normalized, database_features, top_k=5)
print("Top similar images indices:", top_indices)

/var/folders/jn/wrxkscs132b57q3hp4ch7ffc0000gn/T/ipykernel_68206/1084561815.py:15: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNet(weights='imagenet', include_top=False, pooling='avg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
Top similar images indices: [('1.jpg', 0.9999999999999982), ('1406.jpg', 0.5682614106976155), ('321.jpg', 0.566913204399145), ('49.jpg', 0.5373345497847712), ('1236.jpg', 0.5310489255815113)]


In [21]:
import pickle
with open ('index_of_features.pkl', 'rb') as f:
    index_of_features = pickle.load(f)

In [22]:
print(len(index_of_features['1.jpg']))

1024
